## Draw Sankey of UK Energy System

The first step involves importing all the necessary packages to crete the Sankey

In [1]:
import pandas as pd
from sankeyview import *
from sankeyview.jupyter import show_sankey
#from trace_sankey_helpers import show_sample, show_variance, flows_from_trace #, show_sankey

Then we proceed to load the data that we want to draw. In one worksheet we have all the flows, while in the other worksheet we have information about each process in the diagram.

In [3]:
FILENAME = './UK_Energy_Sankey_data.xlsx'
processes = pd.read_excel(FILENAME, sheetname='Processes', index_col=0)
flows = pd.read_excel(FILENAME, sheetname='Flows')


We need to make sure that there is no repetition of flows in the data. Therefore we group and sum all flows.

In [7]:
flows = flows.groupby(['source', 'target', 'type'], as_index=False).sum()

We now create the variable which will be used to plot the diagram

In [8]:
dataset = Dataset(flows, dim_process=processes)

Here we should write down how we want to group each layer of the diagram. This is important also to establish in which order do we the flows to appear. The source grouping is already written down. Try to write one for the end-uses

In [6]:
source_grouping = Partition.Simple(
    'process',
    ['Gas',
    'Liquid fuel',
    'Coal',
    'Nuclear',
    'Solid biomass & Waste',
    'Renewable',
    'Primary Electricity',
    'Heat',
    'Electricity'
])



Here is where we define the nodes and the bundles of the diagram. Essentially we tell the program the names of the nodes, then we tell it in what order they should come (from left to right), then we tell the program from where to where the flows should go.

The information about the "Level" is contained in the processes worksheet. 

In [13]:

default_grouping =  Partition.Simple('process', processes.index.values)


nodes = {
    'inputs':           ProcessGroup(selection="Level == 1", partition=source_grouping),
    'Power Plants':     ProcessGroup(selection="Level == 11",partition=default_grouping ),
    'Devices':          ProcessGroup(selection='Level == 2', partition=default_grouping),
    'Useful':           ProcessGroup(selection='Level == 3', partition=default_grouping) 
}
# Nota Bene
# You sould change the default grouping option once you 
# have written out the grouping options for each layer!

order = [
    ['inputs'],
    ['Power Plants'],
    ['Devices'],
    ['Useful']
]

bundles = [
    Bundle('inputs', 'Devices'),
    Bundle('inputs', 'Power Plants'),
    Bundle('Power Plants', 'Devices'),
    Bundle('Power Plants', 'Useful'),
    Bundle('Devices','Useful')
]


Once all this information is provided, we can compile the variable which generates the structure of the Sankey.

In [10]:
vd = SankeyDefinition(nodes, bundles, order,flow_partition= dataset.partition('type'))

Before we draw the sankey, we need to tell it what colours we want each flow to be. That can be done by defining a Palette. A fwe colours are determined aleady. Add a few more to make the Sankey more interesting.

In [11]:
from collections import defaultdict
palette = defaultdict(lambda: '#999')
palette['Coal'] = 'black'
palette['Electricity'] = 'yellow'


Now we are ready to plot the sankey.

In [14]:
show_sankey(vd, dataset, palette=palette)
# If you want to save the Sankey to the local directory. Just add the 

You can adjust the size and shape of the diagram by definining a height and width ( in pixels)

In [15]:
show_sankey(vd, dataset, width=900, height=600, palette=palette)

If you want to add save the Sankey to the local directory. You can, using the code below. The snankey can be saved in .svg and .png format

In [ ]:
show_sankey(vd, dataset, width=900, height=600,palette=palette).auto_save_svg('UK_Sankey.svg')
show_sankey(vd, dataset, width=900, height=600,palette=palette).auto_save_png('UK_Sankey.png')
